In [13]:
import pandas as pd
import numpy as np

In [4]:
user_data = pd.read_csv('C:/Users/Diya/OneDrive/Desktop/Mini Project/Datasets/fake_transactional_data_24.csv')
user_df = pd.DataFrame(user_data)

In [5]:
user_df.head(10)

,from_totally_fake_account,monopoly_money_amount,to_randomly_generated_account,not_happened_yet_date
0,10371.0,4.00,CINEMA,01/01/2025
1,88339.0,2.40,40544,01/01/2025
2,18555.0,2.40,85149,01/01/2025
3,18555.0,4.10,HIPSTER_COFFEE_SHOP,01/01/2025
4,80792.0,1.95,18555,01/01/2025
5,18555.0,4.45,TOTALLY_A_REAL_COFFEE_SHOP,01/01/2025
6,18555.0,1.45,COFFEE_SHOP,01/01/2025
7,18555.0,5.00,CAFE,01/01/2025
8,41378.0,2.55,COFFEE_SHOP,01/01/2025
9,41378.0,1.80,COFFEE_SHOP,01/01/2025


In [6]:
print(user_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10148280 entries, 0 to 10148279
Data columns (total 4 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   from_totally_fake_account      float64
 1   monopoly_money_amount          float64
 2   to_randomly_generated_account  object 
 3   not_happened_yet_date          object 
dtypes: float64(2), object(2)
memory usage: 309.7+ MB
None


In [1]:
pip install rfm

  Using cached rfm-1.0.9-py3-none-any.whl (7.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
from rfm import rfm

In [19]:
user_df['not_happened_yet_date'] = pd.to_datetime(user_df['not_happened_yet_date'])

In [28]:
# Calculate RFM metrics
max_transaction_date = user_df['not_happened_yet_date'].max()  # Maximum transaction date in the dataset

In [29]:
# Calculate RFM metrics
rfm = user_df.groupby('from_totally_fake_account').agg({
    'not_happened_yet_date': lambda x: (max_transaction_date - x.max()).days,  # Calculate recency based on max_transaction_date
    'to_randomly_generated_account': 'count',
    'monopoly_money_amount': 'sum'
})

In [30]:
# Rename columns
rfm.rename(columns={
    'not_happened_yet_date': 'Recency',
    'to_randomly_generated_account': 'Frequency',
    'monopoly_money_amount': 'Monetary'
}, inplace=True)


In [31]:

# Segment customers
quantiles = rfm.quantile(q=[0.25, 0.50, 0.75])

def rfm_score(x, p, d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

In [32]:
rfm['R'] = rfm['Recency'].apply(rfm_score, args=('Recency', quantiles))
rfm['F'] = rfm['Frequency'].apply(rfm_score, args=('Frequency', quantiles))
rfm['M'] = rfm['Monetary'].apply(rfm_score, args=('Monetary', quantiles))

In [33]:
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis=1)

In [34]:
print(rfm)

                           Recency  Frequency  Monetary  R  F  M RFM_Segment  \
from_totally_fake_account                                                      
1000.0                           0       1710  28698.51  4  1  2         412   
1002.0                           0       1043  15622.97  4  3  4         434   
1006.0                           0       1841  18778.19  4  1  3         413   
1018.0                           1        805  15612.21  1  4  4         144   
1053.0                           0       1319  19444.57  4  2  3         423   
...                            ...        ...       ... .. .. ..         ...   
100931.0                         0       1664  22332.81  4  1  2         412   
100934.0                         1       1111  18579.25  1  3  3         133   
100947.0                         0       1586  20783.20  4  1  3         413   
100986.0                         0       1034  15314.23  4  3  4         434   
100988.0                         0      

In [35]:
# Sort by Recency score in descending order
rfm_sorted_by_recency = rfm.sort_values(by='Recency', ascending=False)

# Sort by Frequency score in descending order
rfm_sorted_by_frequency = rfm.sort_values(by='Frequency', ascending=False)

# Sort by Monetary score in descending order
rfm_sorted_by_monetary = rfm.sort_values(by='Monetary', ascending=False)

print("Sorted by Recency score:")
print(rfm_sorted_by_recency)

print("\nSorted by Frequency score:")
print(rfm_sorted_by_frequency)

print("\nSorted by Monetary score:")
print(rfm_sorted_by_monetary)


Sorted by Recency score:
                           Recency  Frequency  Monetary  R  F  M RFM_Segment  \
from_totally_fake_account                                                      
1217.0                           4        895  17761.23  1  4  3         143   
57276.0                          4        803  13933.83  1  4  4         144   
45079.0                          4        794  17877.99  1  4  3         143   
43858.0                          4       1126  19890.51  1  3  3         133   
68755.0                          3        645  13341.67  1  4  4         144   
...                            ...        ...       ... .. .. ..         ...   
37263.0                          0        807  30829.22  4  4  1         441   
37260.0                          0       1497  22726.87  4  2  2         422   
37249.0                          0       1257  18008.06  4  2  3         423   
37245.0                          0        986  18716.84  4  4  3         443   
100988.0       